In [ ]:
!pip install pandas numpy scikit-learn xgboost lightgbm matplotlib


#Imports

In [ ]:

import pandas as pd
import datetime
from random import randint
from time import sleep
import os
import requests
import numpy as np
import matplotlib.pyplot as plt

#**NFL**

 Currating data from baseline Home win rate. Using data from 'https://www.pro-football-reference.com/years/' (2019-2025) to obtain a prediction. Below is the data cleaning for this prediction and Helper Functions


## Data Manipulation & Data Cleaning

In [ ]:
#Mounting Google Drive
from google.colab import drive
drive.mount('/content/drive')

#Configure which seasons to pull
START_SEASON = 2019
END_SEASON = 2024

# PFR home/away marker column (contains '@' if the Winner played AWAY)
HA_COL = 'Unnamed: 5'

print(f"Configured seasons: {START_SEASON}–{END_SEASON}, HA_COL={HA_COL}")



Mounted at /content/drive
Configured seasons: 2019–2024, HA_COL=Unnamed: 5


In [ ]:
# Helper Functions
def assign_home_away_from_marker(row, ha_col: str) -> pd.Series:
    """
    Reconstruct home/away teams and scores using the PFR '@' marker:
      - If row[ha_col] == '@'  → Winner/tie was AWAY ⇒ Loser/tie is HOME
      - Else (blank/NaN)       → Winner/tie was HOME ⇒ Loser/tie is AWAY
    Returns a Series: home_team, away_team, home_score, away_score
    """
    winner = row['Winner/tie']
    loser  = row['Loser/tie']
    pts_w  = row['PtsW']
    pts_l  = row['PtsL']

    winner_is_away = (str(row.get(ha_col, '')).strip() == '@')

    if winner_is_away:
        home_team, away_team = loser, winner
        home_score, away_score = pts_l, pts_w
    else:
        home_team, away_team = winner, loser
        home_score, away_score = pts_w, pts_l

    return pd.Series(
        [home_team, away_team, home_score, away_score],
        index=['home_team', 'away_team', 'home_score', 'away_score']
    )

def label_season_type_from_week(week_value) -> str:
    """
    'Week' is numeric for regular season (1..18) and a string label for playoffs
    ('WildCard', 'Division', 'ConfChamp', 'SuperBowl').
    """
    try:
        _ = int(str(week_value))
        return 'regular'
    except (TypeError, ValueError):
        return 'postseason'

def to_int_or_na(week_value):
    """Convert 'Week' to int when possible; otherwise return <NA> (for playoffs)."""
    try:
        return int(str(week_value))
    except (TypeError, ValueError):
        return pd.NA




In [ ]:
# Fetch seasons and summarize raw tables
all_nfl_games_raw = []
summary = []

for year in range(START_SEASON, END_SEASON + 1):
    url = f"https://www.pro-football-reference.com/years/{year}/games.htm"
    print(f"\nFetching: {url}")
    dfs = pd.read_html(url)
    print(f"  -> found {len(dfs)} HTML table(s): {[df.shape for df in dfs]}")
    df_raw = dfs[0].copy()
    df_raw['season'] = year
    all_nfl_games_raw.append(df_raw)
    summary.append({
        "season": year,
        "num_tables": len(dfs),
        "shape": dfs[0].shape,
        "columns": list(dfs[0].columns)
    })

print("\n=== Summary (first table per season) ===")
for s in summary:
    print(f"{s['season']}: shape={s['shape']}, columns={s['columns']}")


Fetching: https://www.pro-football-reference.com/years/2019/games.htm
  -> found 1 HTML table(s): [(284, 14)]

Fetching: https://www.pro-football-reference.com/years/2020/games.htm
  -> found 1 HTML table(s): [(286, 14)]

Fetching: https://www.pro-football-reference.com/years/2021/games.htm
  -> found 1 HTML table(s): [(303, 14)]

Fetching: https://www.pro-football-reference.com/years/2022/games.htm
  -> found 1 HTML table(s): [(302, 14)]

Fetching: https://www.pro-football-reference.com/years/2023/games.htm
  -> found 1 HTML table(s): [(303, 14)]

Fetching: https://www.pro-football-reference.com/years/2024/games.htm
  -> found 1 HTML table(s): [(303, 14)]

=== Summary (first table per season) ===
2019: shape=(284, 14), columns=['Week', 'Day', 'Date', 'Time', 'Winner/tie', 'Unnamed: 5', 'Loser/tie', 'Unnamed: 7', 'PtsW', 'PtsL', 'YdsW', 'TOW', 'YdsL', 'TOL']
2020: shape=(286, 14), columns=['Week', 'Day', 'Date', 'Time', 'Winner/tie', 'Unnamed: 5', 'Loser/tie', 'Unnamed: 7', 'PtsW', 'P

In [ ]:
cleaned_frames = []

for df_raw in all_nfl_games_raw:
    # Keep only the columns we need
    keep_cols = ['Week', 'Date', 'Winner/tie', HA_COL, 'Loser/tie', 'PtsW', 'PtsL', 'season']
    df = df_raw[keep_cols].copy()

    # Drop repeated header rows
    df = df[df['Week'].astype(str).str.lower() != 'week'].copy()

    # Parse / coerce types
    df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
    df['PtsW'] = pd.to_numeric(df['PtsW'], errors='coerce')
    df['PtsL'] = pd.to_numeric(df['PtsL'], errors='coerce')

    # Build home/away & scores using the marker
    df[['home_team', 'away_team', 'home_score', 'away_score']] = df.apply(
        assign_home_away_from_marker, axis=1, ha_col=HA_COL
    )

    # Tag season type and numeric week
    df['season_type'] = df['Week'].apply(label_season_type_from_week)
    df['week_num']    = df['Week'].apply(to_int_or_na)

    # Drop rows without real scores (future schedule placeholders / malformed rows)
    df = df.dropna(subset=['home_score', 'away_score'])

    cleaned_frames.append(df)

# Combine all seasons into one tidy frame
games = pd.concat(cleaned_frames, ignore_index=True)

# Reorder columns for readability
games = games[
    [
        'season', 'season_type', 'Week', 'week_num', 'Date',
        'home_team', 'away_team', 'home_score', 'away_score',
        'Winner/tie', 'Loser/tie', 'PtsW', 'PtsL', HA_COL
    ]
]

print("Done. Rows:", len(games))
games.head(25)

Done. Rows: 1675


,season,season_type,Week,week_num,Date,home_team,away_team,home_score,away_score,Winner/tie,Loser/tie,PtsW,PtsL,Unnamed: 5
0,2019,regular,1,1,2019-09-05,Chicago Bears,Green Bay Packers,3.0,10.0,Green Bay Packers,Chicago Bears,10.0,3.0,@
1,2019,regular,1,1,2019-09-08,Minnesota Vikings,Atlanta Falcons,28.0,12.0,Minnesota Vikings,Atlanta Falcons,28.0,12.0,NaN
2,2019,regular,1,1,2019-09-08,New York Jets,Buffalo Bills,16.0,17.0,Buffalo Bills,New York Jets,17.0,16.0,@
3,2019,regular,1,1,2019-09-08,Carolina Panthers,Los Angeles Rams,27.0,30.0,Los Angeles Rams,Carolina Panthers,30.0,27.0,@
4,2019,regular,1,1,2019-09-08,Cleveland Browns,Tennessee Titans,13.0,43.0,Tennessee Titans,Cleveland Browns,43.0,13.0,@
5,2019,regular,1,1,2019-09-08,Jacksonville Jaguars,Kansas City Chiefs,26.0,40.0,Kansas City Chiefs,Jacksonville Jaguars,40.0,26.0,@
6,2019,regular,1,1,2019-09-08,Miami Dolphins,Baltimore Ravens,10.0,59.0,Baltimore Ravens,Miami Dolphins,59.0,10.0,@
7,2019,regular,1,1,2019-09-08,Philadelphia Eagles,Washington Redskins,32.0,27.0,Philadelphia Eagles,Washington Redskins,32.0,27.0,NaN
8,2019,regular,1,1,2019-09-08,Seattle Seahawks,Cincinnati Bengals,21.0,20.0,Seattle Seahawks,Cincinnati Bengals,21.0,20.0,NaN
9,2019,regular,1,1,2019-09-08,Los Angeles Chargers,Indianapolis Colts,30.0,24.0,Los Angeles Chargers,Indianapolis Colts,30.0,24.0,NaN


Save the cleaned per-game table (for reuse)

In [ ]:
DATA_NFL_DIR = "/content/drive/MyDrive/NFL_Project/reports"
os.makedirs(DATA_NFL_DIR, exist_ok=True)
clean_path = os.path.join(DATA_NFL_DIR, f"games_clean_{START_SEASON}_{END_SEASON}.csv")
games.to_csv(clean_path, index=False)
print(f"Saved clean dataset to: {clean_path}")

Saved clean dataset to: /content/drive/MyDrive/NFL_Project/reports/games_clean_2019_2024.csv


In [ ]:
DATA_NFL_DIR = "/content/drive/MyDrive/NFL_Project/reports"
os.makedirs(DATA_NFL_DIR, exist_ok=True)

reg = games[games['season_type'] == 'regular'].copy()

#ensure numeric(should always be numeric)
reg['home_score'] = pd.to_numeric(reg['home_score'], errors='coerce')
reg['away_score'] = pd.to_numeric(reg['away_score'], errors= 'coerce')

#flags
reg['home_win_excl_ties'] = pd.to_numeric(reg['home_score'] > reg['away_score']).astype(int)
reg['is_tie'] = (reg['home_score'] == reg['away_score'])

#count ties as 0.5
reg['home_win_incl_ties'] = (
    reg['home_win_excl_ties'].astype(float) + 0.5 * reg['is_tie'].astype(float)
)

#League-wide baseline (regular season)
overall_excl_ties = reg.loc[~reg['is_tie'], 'home_win_excl_ties'].mean()
overall_incl_ties = reg['home_win_incl_ties'].mean()

print("=== League-wide Home Win Rate (Regular Season) ===")
print(f"Exclude ties: {overall_excl_ties:.3%}")
print(f"Include ties: {overall_incl_ties:.3%}")
print()

#By-season Baseline
by_season = (
    #group based on season
    reg.groupby('season')
    .apply(lambda d: pd.Series({
        #number of games
        'games': len(d),
        #number of ties
        'ties': int(d['is_tie'].sum()),
        # win or lose rate exclusing ties aka 1 or 0.
        'home_win_rate_excl_ties': d.loc[~d['is_tie'], 'home_win_excl_ties'].mean() if (~d['is_tie']).any() else float('nan'),
        #average the win ties and loss
        'home_win_rate_incl_ties': d['home_win_incl_ties'].mean()
    }))
    .reset_index()
    .sort_values('season')
)

print("=== Home win Rate by Season(regular season) === ")
display(by_season.head(20))

#By-team (home games only) baseline
by_team_home= (
  reg.groupby('home_team')
    .apply(lambda d: pd.Series({

        'home_games': len(d),
        'ties' : int(d['is_tie'].sum()),
        'home_win_rate_excl_ties' : d.loc[~d['is_tie'], 'home_win_excl_ties'].mean() if (~d['is_tie']).any() else float('nan'),
        'home_win_rate_incl_ties' : d['home_win_incl_ties'].mean()
    }))

    .reset_index()
    .sort_values('home_win_rate_excl_ties', ascending=False)
)

print("== Home Win Rate by Home Team (regular season) ==")
display(by_team_home.head(30))

#Save output to /content/reports
overall_df = pd.DataFrame({
    'metric': ['home_win_rate_excl_ties', 'home_win_rate_incl_ties'],
    'value': [overall_excl_ties, overall_incl_ties]
})
overall_path = os.path.join(DATA_NFL_DIR,'home_win_rate_overall.csv')
by_season_path = os.path.join(DATA_NFL_DIR, 'home_win_rate_by_season.csv')
by_team_home_path = os.path.join(DATA_NFL_DIR, 'home_win_rate_by_team_home.csv')

overall_df.to_csv(overall_path, index=False)
by_season.to_csv(by_season_path, index=False)
by_team_home.to_csv(by_team_home_path, index=False)

print("\nSaved:")
print(f"  - {overall_path}")
print(f"  - {by_season_path}")
print(f"  - {by_team_home_path}")


=== League-wide Home Win Rate (Regular Season) ===
Exclude ties: 53.074%
Include ties: 53.064%

=== Home win Rate by Season(regular season) === 


/tmp/ipython-input-890914950.py:32: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda d: pd.Series({


,season,games,ties,home_win_rate_excl_ties,home_win_rate_incl_ties
0,2019,256.0,1.0,0.517647,0.517578
1,2020,256.0,1.0,0.498039,0.498047
2,2021,272.0,1.0,0.516605,0.516544
3,2022,271.0,2.0,0.561338,0.560886
4,2023,272.0,0.0,0.555147,0.555147
5,2024,272.0,0.0,0.533088,0.533088


== Home Win Rate by Home Team (regular season) ==


/tmp/ipython-input-890914950.py:52: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda d: pd.Series({


,home_team,home_games,ties,home_win_rate_excl_ties,home_win_rate_incl_ties
3,Buffalo Bills,50.0,0.0,0.780000,0.780000
11,Green Bay Packers,50.0,0.0,0.760000,0.760000
15,Kansas City Chiefs,50.0,0.0,0.760000,0.760000
2,Baltimore Ravens,50.0,0.0,0.680000,0.680000
26,Philadelphia Eagles,50.0,1.0,0.653061,0.650000
27,Pittsburgh Steelers,50.0,1.0,0.653061,0.650000
20,Minnesota Vikings,50.0,0.0,0.640000,0.640000
19,Miami Dolphins,50.0,0.0,0.640000,0.640000
8,Dallas Cowboys,50.0,0.0,0.640000,0.640000
25,Oakland Raiders,8.0,0.0,0.625000,0.625000



Saved:
  - /content/drive/MyDrive/NFL_Project/reports/home_win_rate_overall.csv
  - /content/drive/MyDrive/NFL_Project/reports/home_win_rate_by_season.csv
  - /content/drive/MyDrive/NFL_Project/reports/home_win_rate_by_team_home.csv


## Incorporating International Games(Data Integraty)
- No Dataset was found therefor I manual inputed the games on a CSV file.


In [ ]:
DATA_NFL_DIR = "/content/drive/MyDrive/NFL_Project/reports"

#file paths
games_path = os.path.join(DATA_NFL_DIR, "games_clean_2019_2024.csv")
int_games_path = os.path.join(DATA_NFL_DIR, "international_games.csv")
output_path = os.path.join(DATA_NFL_DIR, "games_clean_2019_2024_international.csv")

#load data
games = pd.read_csv(games_path, parse_dates=["Date"])
int_games = pd.read_csv(int_games_path, parse_dates=["Date"])

#Ensure consistent data type for merge keys
games["Season"] = games["season"].astype(int)
int_games["Season"] = int_games["Season"].astype(int)

#clean up potential formatting issues (extra space, differnt case)
for col in ["home_team", "away_team"]:
  games[col] = games[col].astype(str).str.strip()
  int_games[col] = int_games[col].astype(str).str.strip()

#Perform Merge
merged = games.merge(
    int_games.assign(is_international = 1),
    on = ["Season", "Date", "home_team", "away_team"],
    how = "left"
)

#Fill non-matches with 0(domestic game)
merged["is_international"] = merged["is_international"].fillna(0).astype(int)

#Quick verification
print("=== Merge complete ===")
print(f"  - International game tagged: {merged['is_international'].sum()}")
print(f"  - Domestic games: {len(merged) - merged['is_international'].sum()}")
print(f"\nSample of tagged rows: ")
display(merged[merged["is_international"] == 1][["season", "Date", "home_team", "away_team", "is_international"]].head())

#Check per-season count
display(merged.groupby("season")["is_international"].sum())

#Saving file
merged.to_csv(output_path, index=False)
print(F"\n Saved updated file with international flag -> {output_path}")



=== Merge complete ===
  - International game tagged: 22
  - Domestic games: 1653

Sample of tagged rows: 


,season,Date,home_team,away_team,is_international
68,2019,2019-10-06,Oakland Raiders,Chicago Bears,1
79,2019,2019-10-13,Tampa Bay Buccaneers,Carolina Panthers,1
112,2019,2019-10-27,Los Angeles Rams,Cincinnati Bengals,1
122,2019,2019-11-03,Jacksonville Jaguars,Houston Texans,1
161,2019,2019-11-18,Los Angeles Chargers,Kansas City Chiefs,1


,is_international
season,
2019,5
2020,0
2021,2
2022,5
2023,5
2024,5



 Saved updated file with international flag -> /content/drive/MyDrive/NFL_Project/reports/games_clean_2019_2024_international.csv


## Incorporating HFA metrics:

Home Field Advantage (HFA) Calculation

The home-field advantage (HFA) was derived empirically from observed win rates between 2019–2024.


The equivalent ELO point advantage was calculated using:

- Δ𝑅= −400 × log⁡10((1/Pwin)-1)

This converts a home-win probability Pwin into its ELO rating offset.

We apply a hierarchical structure:

- Team-specific HFA (from team home win rates)

- Seasonal HFA (from year-level win rates)

- League-wide average HFA (fallback)

- International games → 0

The resulting column home_field_advantage_pts is used in all ELO calculation

In [ ]:
#File location
DATA_NFL_DIR = "/content/drive/MyDrive/NFL_Project/reports"

#File path
path_Games = os.path.join(DATA_NFL_DIR, "games_clean_2019_2024_international.csv")
path_Overall = os.path.join(DATA_NFL_DIR, "home_win_rate_overall.csv")
path_Season = os.path.join(DATA_NFL_DIR, "home_win_rate_by_season.csv")
path_Team = os.path.join(DATA_NFL_DIR, "home_win_rate_by_team_home.csv")

#output path
output_path = os.path.join(DATA_NFL_DIR, "games_clean_2019_2024_international_HFA.csv")

def p_to_elo_adv(p_win : float) -> float:
  """
  Convert a home win probability into an ELO rating offset (difference R in points)
  using the inverse of ELO expectation formula
  ΔR = −400 × log⁡10((1/Pwin)-1)

  """

  #clamp to avoid log/inf if input is 0 or 1
  p = float(np.clip(p_win, 1e-6, 1- 1e-6))
  return -400 * np.log10(1.0/p - 1.0)

#load main games file
games = pd.read_csv(path_Games, parse_dates=["Date"])
games["season"] = games ["season"].astype(int)
for c in ["home_team","away_team"]:
  games[c] = games[c].astype(str).str.strip()


#load HFA sources
overall = pd.read_csv(path_Overall)
by_season = pd.read_csv(path_Season)
by_team = pd.read_csv(path_Team)

#Standardize column names (lowercase, trim)
overall.columns = [c.strip().lower() for c in overall.columns]
by_season.columns = [c.strip().lower() for c in by_season.columns]
by_team.columns = [c.strip().lower() for c in by_team.columns]

#Extract a probability column from a table
def pick_prob_column(df, preferred_order = ("home_win_rate_excl_ties", "home_win_rate_incl_ties","home_win_rate", "p_home_win", "pct", "value")):
  cols = list(df.columns)

  #try any of the first float-ish column that looks like a rate
  for name in preferred_order:
    if name in cols:
      return name
  #otherwise try to find the first float-ish column that looks like a rate
  for c in cols:
    if df[c].dtype.kind in "fc":
      #Heuristic. values between 0 and 1 (prob) or 0..100(%)
      sample = df[c].dropna()
      if not sample.empty:
        q = sample.quantile([0.05,0.95]).values
        if(0 <= q[0] <= 1 and 0<=q[1] <= 1) or (0 <= q[0] <= 100 and 0 <= q[1] <= 100):
          return c
  raise ValueError("Could not find a home-win-rate column in the provided dataframe")

#OVERALL can be a 2-row table with 'metric'/'value
if "metric" in overall.columns:
  row = overall.loc[overall["metric"].str.contains("excl", case=False, na = False)]
  if row.empty:
    row = overall.iloc[[0]]
  overall_prob = row[pick_prob_column(overall)].iloc[0]
else:
  #simple-row or simple table: take first numeric colum
  overall_prob = overall[pick_prob_column(overall)].iloc[0]

#convert % to proportion if needed
overall_prob = float(overall_prob)
if overall_prob >= 1.0:
  overall_prob = overall_prob/100.0

#By Season: expect a 'Season; column and a win-rate column
season_prob_col = pick_prob_column(by_season)
by_season = by_season.copy()
by_season["season"] = by_season["season"].astype(int)
#normalize to protion if given a percentage
if by_season[season_prob_col].max() > 1.0:
  by_season[season_prob_col] = by_season[season_prob_col]/100.0

#by team: expect 'home_team' and win-rate col
team_prob_col = pick_prob_column(by_team)
by_team = by_team.copy()
by_team["home_team"] = by_team["home_team"].astype(str).str.strip()
if by_team[team_prob_col].max() > 1.0:
  by_team[team_prob_col] = by_team[team_prob_col]/100.0

#League-wide ELO HFA(Home Field Advantage)
elo_hfa_overall = p_to_elo_adv(overall_prob)

#per season elo HFA
hfa_season = by_season[["season", season_prob_col]].rename(columns={season_prob_col: "p_home_win_season"})

#per-team elo HFA (home team prespective)
hfa_team = by_team[["home_team", team_prob_col]].rename(columns={team_prob_col:"p_home_win_team"})
hfa_team["elo_hfa_pts_team"] = hfa_team["p_home_win_team"].apply(p_to_elo_adv)
print(f"League-wide ELO HFA (points): {elo_hfa_overall:.2f}")
display(hfa_season.head(6))
display(hfa_team.head(10))






League-wide ELO HFA (points): 21.39


,season,p_home_win_season
0,2019,0.517647
1,2020,0.498039
2,2021,0.516605
3,2022,0.561338
4,2023,0.555147
5,2024,0.533088


,home_team,p_home_win_team,elo_hfa_pts_team
0,Buffalo Bills,0.780000,219.868769
1,Green Bay Packers,0.760000,200.240940
2,Kansas City Chiefs,0.760000,200.240940
3,Baltimore Ravens,0.680000,130.943574
4,Philadelphia Eagles,0.653061,109.880423
5,Pittsburgh Steelers,0.653061,109.880423
6,Minnesota Vikings,0.640000,99.950989
7,Miami Dolphins,0.640000,99.950989
8,Dallas Cowboys,0.640000,99.950989
9,Oakland Raiders,0.625000,88.739500


Team-Primary HFA Hierarchy
Each game’s home-field advantage (HFA) in Elo points is determined primarily by the home team’s long-term home win rate.
A small contextual adjustment (β = 0.3) is applied based on the league-wide home-field advantage for that season relative to the multi-year league average.
This approach ensures the model captures venue-specific effects while accounting for yearly fluctuations (e.g., pandemic years, travel effects).

Formula:

HFA_game = HFA_team + beta(HFA_season - HFA_league)


In [ ]:
# ==========================================
# Name canonicalization
# ==========================================

# Define canonical mapping → use today's official names
NAME_MAP = {
    "Washington Redskins": "Washington Commanders",
    "Washington Football Team": "Washington Commanders",
    # Move-related (kept for integrity going forward)
    "Oakland Raiders": "Las Vegas Raiders",
    # Safety aliases that sometimes show up (not expected 2019–2024 but harmless)
    "St. Louis Rams": "Los Angeles Rams",
    "San Diego Chargers": "Los Angeles Chargers",
}

def _apply_team_map(s):
    return s.astype(str).str.strip().replace(NAME_MAP)

#Apply to the main games dataframe (both home and away)
games["home_team"] = _apply_team_map(games["home_team"])
games["away_team"] = _apply_team_map(games["away_team"])

# Apply to by_team table (this table’s “home_team” is the grouping key)
if "home_team" in by_team.columns:
    by_team["home_team"] = _apply_team_map(by_team["home_team"])

#    consolidate any duplicate rows created by renaming
#    We’ll recompute the team-level home win rate by averaging rows that now share the same name.
#    If your by_team table already has a count column (e.g., 'n_games'), replace the mean with a weighted mean.
team_prob_col = team_prob_col  # from earlier cell
if by_team.duplicated(subset=["home_team"]).any():
    # If you have a count column like 'games' or 'n', do a weighted average instead:
    count_col_candidates = [c for c in by_team.columns if c.lower() in ("games","n","n_games","count","home_games")]
    if count_col_candidates:
        wcol = count_col_candidates[0]
        by_team = (
            by_team
            .groupby("home_team", as_index=False)
            .apply(lambda d: pd.Series({
                team_prob_col: np.average(d[team_prob_col], weights=d[wcol]),
                wcol: d[wcol].sum()
            }))
            .reset_index(drop=True)
        )
    else:
        # No counts available → simple mean
        by_team = by_team.groupby("home_team", as_index=False)[team_prob_col].mean()

# Quick validation: ensure only the canonical names remain
for col in ["home_team", "away_team"]:
    bad = games[col][games[col].isin(NAME_MAP.keys())].unique()
    if len(bad):
        print(f"Still found legacy names in games[{col}]:", bad)
    else:
        print(f"games[{col}] canonicalized")

if "home_team" in by_team.columns:
    bad_bt = by_team["home_team"][by_team["home_team"].isin(NAME_MAP.keys())].unique()
    if len(bad_bt):
        print("Still found legacy names in by_team:", bad_bt)
    else:
        print("by_team canonicalized")

# (Optional) Persist the canonicalized games file so downstream steps read the same labels
#    — only do this if you want the on-disk CSV updated now:
# games.to_csv(path_Games, index=False)
# print(f"💾 Saved canonicalized games back to: {path_Games}")


games[home_team] canonicalized
games[away_team] canonicalized
by_team canonicalized


In [ ]:

#Convert observed win rates -> elo-point HFAS
#league overall (fallback)
elo_hfa_overall = p_to_elo_adv(overall_prob)

#-per season:map season -> elo points
hfa_season = by_season[["season", season_prob_col]].rename(columns = {season_prob_col: "p_home_win_season"}).copy()
hfa_season["elo_hfa_pts_season"] = hfa_season["p_home_win_season"].apply(p_to_elo_adv)

#-per team
hfa_team = by_team[["home_team", team_prob_col]].rename(columns = {team_prob_col: "p_home_win_team"}).copy()
hfa_team["elo_hfa_pts_team"] = hfa_team["p_home_win_team"].apply(p_to_elo_adv)

#Merge per-team HFA first
g = games.merge(hfa_team[["home_team", "elo_hfa_pts_team"]], on = "home_team", how="left")

#merge season (for contextual adjustment)
g = g.merge(hfa_season[["season", "elo_hfa_pts_season"]], on = "season", how = "left")

#Fill missing values
g["elo_hfa_pts_team_filled"] = g["elo_hfa_pts_team"].fillna(elo_hfa_overall)
g["elo_hfa_pts_season_filled"] = g["elo_hfa_pts_season"].fillna(elo_hfa_overall)

#Blending parameter (season context weight)
BETA_SEASON_CONTEXT = 0.30 #smaller than 1 because team dominates

#Team-primary final HFA
g["home_field_advantage_pts"] = (g["elo_hfa_pts_team_filled"] + BETA_SEASON_CONTEXT * (g["elo_hfa_pts_season_filled"] - elo_hfa_overall))

#international (neutral-site)
if "is_international" in g.columns:
  g.loc[g["is_international"]==1, "home_field_advantage_pts"] = 0.0

#safety caps to avoid pathological values
g["home_field_advantage_pts"] = g["home_field_advantage_pts"].clip(-10, 90)

#validation
print(f"League-wide HFA (fallback) in Elo points: {elo_hfa_overall:.2f}")
print("\nHome-field advantage (Elo pts) summary after blending & overrides:")
print(g["home_field_advantage_pts"].describe())

print("\nSample of rows (season/home_team/HFA):")
display(g.sample(min(8, len(g)))[
    ["season","Date","home_team","away_team","is_international",
     "home_field_advantage_pts","elo_hfa_pts_team","elo_hfa_pts_season"]
])

g.to_csv(output_path, index=False)
print(f"\nSaved HFA-enriched dataset -> {output_path}")



League-wide HFA (fallback) in Elo points: 21.39

Home-field advantage (Elo pts) summary after blending & overrides:
count    1675.000000
mean       32.220565
std        42.992732
min       -10.000000
25%       -10.000000
50%        19.031580
75%        90.000000
max        90.000000
Name: home_field_advantage_pts, dtype: float64

Sample of rows (season/home_team/HFA):


,season,Date,home_team,away_team,is_international,home_field_advantage_pts,elo_hfa_pts_team,elo_hfa_pts_season
827,2022,2022-09-11,Detroit Lions,Philadelphia Eagles,0,-7.469702,-13.904843,42.837976
1058,2022,2022-12-25,Los Angeles Rams,Denver Broncos,0,76.871645,70.436504,42.837976
1003,2022,2022-12-04,Baltimore Ravens,Denver Broncos,0,90.000000,130.943574,42.837976
506,2020,2020-12-28,New England Patriots,Buffalo Bills,0,-10.000000,-41.894140,-1.362499
1275,2023,2023-11-26,Cincinnati Bengals,Pittsburgh Steelers,0,12.218244,7.091507,38.476631
388,2020,2020-11-08,Minnesota Vikings,Detroit Lions,0,90.000000,99.950989,-1.362499
189,2019,2019-12-01,Kansas City Chiefs,Las Vegas Raiders,0,90.000000,200.240940,12.267528
1065,2022,2023-01-01,Washington Commanders,Cleveland Browns,0,-10.000000,-85.043237,42.837976



Saved HFA-enriched dataset -> /content/drive/MyDrive/NFL_Project/reports/games_clean_2019_2024_international_HFA.csv


## HFA verification.


In [ ]:
# Load the file you uploaded
path = "/content/drive/MyDrive/NFL_Project/reports/games_clean_2019_2024_international_HFA.csv"
df = pd.read_csv(path, parse_dates=["Date"])

# Basic overview
print("=== Dataset Info ===")
print(df.info())
print("\n=== Column Preview ===")
print(df.columns.tolist())

# Quick sanity checks on HFA column
print("\n=== HFA (Elo points) summary ===")
print(df["home_field_advantage_pts"].describe())

# Verify that international games were zeroed out
intl_check = df.loc[df["is_international"] == 1, "home_field_advantage_pts"]
print(f"\nInternational games count: {len(intl_check)}")
print(f"Non-zero values in international games: {(intl_check != 0).sum()}")

# Check a few random rows to ensure different teams & seasons
print("\n=== Random Sample of 10 Games ===")
display(df.sample(10)[["season","Date","home_team","away_team",
                       "is_international","home_field_advantage_pts"]])

# Optional: Group summaries to see if team-based HFA patterns make sense
team_summary = df.groupby("home_team")["home_field_advantage_pts"].mean().sort_values(ascending=False)
print("\n=== Average HFA by Home Team (Top 10) ===")
display(team_summary.head(10))

print("\n=== Average HFA by Home Team (Bottom 10) ===")
display(team_summary.tail(10))

# Check average HFA by season to confirm the small contextual adjustments
season_summary = df.groupby("season")["home_field_advantage_pts"].mean()
print("\n=== Average HFA by Season ===")
display(season_summary)

=== Dataset Info ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1675 entries, 0 to 1674
Data columns (total 21 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   season                     1675 non-null   int64         
 1   season_type                1675 non-null   object        
 2   Week                       1675 non-null   object        
 3   week_num                   1599 non-null   float64       
 4   Date                       1675 non-null   datetime64[ns]
 5   home_team                  1675 non-null   object        
 6   away_team                  1675 non-null   object        
 7   home_score                 1675 non-null   float64       
 8   away_score                 1675 non-null   float64       
 9   Winner/tie                 1675 non-null   object        
 10  Loser/tie                  1675 non-null   object        
 11  PtsW                       1675 non-null   float

,season,Date,home_team,away_team,is_international,home_field_advantage_pts
257,2019,2020-01-04,New England Patriots,Tennessee Titans,0,-10.000000
356,2020,2020-10-19,Buffalo Bills,Kansas City Chiefs,0,90.000000
618,2021,2021-10-17,Carolina Panthers,Minnesota Vikings,0,-10.000000
1171,2023,2023-10-08,Atlanta Falcons,Houston Texans,0,-10.000000
405,2020,2020-11-15,New York Giants,Philadelphia Eagles,0,-10.000000
1570,2024,2024-11-28,Dallas Cowboys,New York Giants,0,90.000000
1420,2024,2024-09-15,Houston Texans,Chicago Bears,0,-10.000000
31,2019,2019-09-16,New York Jets,Cleveland Browns,0,-10.000000
491,2020,2020-12-25,New Orleans Saints,Minnesota Vikings,0,21.029370
843,2022,2022-09-18,New Orleans Saints,Tampa Bay Buccaneers,0,34.289512



=== Average HFA by Home Team (Top 10) ===


,home_field_advantage_pts
home_team,
Baltimore Ravens,90.000000
Dallas Cowboys,90.000000
Pittsburgh Steelers,90.000000
Miami Dolphins,90.000000
Kansas City Chiefs,88.615385
Buffalo Bills,88.474576
Philadelphia Eagles,88.421053
Green Bay Packers,88.333333
Minnesota Vikings,88.235294



=== Average HFA by Home Team (Bottom 10) ===


,home_field_advantage_pts
home_team,
Arizona Cardinals,-9.800000
Carolina Panthers,-9.800000
Chicago Bears,-9.800000
Atlanta Falcons,-9.800000
Los Angeles Chargers,-9.800000
New England Patriots,-9.803922
Houston Texans,-10.000000
New York Giants,-10.000000
New York Jets,-10.000000



=== Average HFA by Season ===


,home_field_advantage_pts
season,
2019,31.048188
2020,30.213762
2021,31.747298
2022,34.346741
2023,33.306779
2024,32.481372


In [ ]:
!ls -lh "/content/drive/MyDrive/NFL_Project/reports/"

total 417K
-rw------- 1 root root 199K Oct 24 17:59 games_clean_2019_2024.csv
-rw------- 1 root root  181 Oct 24 18:17 games_clean_2019_2024.gsheet
-rw------- 1 root root 214K Oct 24 18:27 games_clean_2019_2024_international.csv
-rw------- 1 root root  370 Oct 24 18:00 home_win_rate_by_season.csv
-rw------- 1 root root 1.5K Oct 24 18:00 home_win_rate_by_team_home.csv
-rw------- 1 root root   99 Oct 24 18:00 home_win_rate_overall.csv
-rw------- 1 root root 1.2K Oct 24 18:22 international_games.csv
-rw------- 1 root root  181 Oct 24 18:22 international_games.gsheet


## MOV & Expected win rate

In [ ]:
IN_PATH = f"{DATA_NFL_DIR}/games_clean_2019_2024_international_HFA.csv"
OUT_PATH = f"{DATA_NFL_DIR}/games_clean_2019_2024_international_HFA_MOV.csv"

try:

  df = g.copy()

except NameError:
  df = pd.read_csv(IN_PATH)

#Columns check
req = ["home_team","away_team","home_score","away_score","home_field_advantage_pts"]
missing = [c for c in req if c not in df.columns]
if missing:
  raise ValueError(f"Missing required columns: {missing}")

#Type check
for c in["home_score","away_score","home_field_advantage_pts"]:
  df[c] = pd.to_numeric(df[c], errors="coerce")

#Enforce HFA = 0 for neutral/internation games
intl_flag_col = None
for cand in ["is_international","is_neutral","neutral_site"]:
  if cand in df.columns:
    intl_flag_col = cand
    break

if intl_flag_col is None:
  #normalize truthy values to bool
  m_intl = df[intl_flag_col].astype(str).str.lower().isin(["1", "true", "t", "yes", "y"])
  df.loc[m_intl, "home_field_advantage_pts"] == 0.0

#Expected home win probability

#pre-ELO placeholder
df["pre_elo_home"] = 1500.0
df["pre_elo_away"] = 1500.0

# E_home = 1/ (1+10 ^ (-((R_home + HFA) - R_away) / 400 ))
rating_diff = (df["pre_elo_home"] + df["home_field_advantage_pts"]) - df["pre_elo_away"]
df["expected_home_win_prob"] = 1.0/(1.0+np.power(10.0, - (rating_diff/400.0)))

#Margin of Victory
df["margin_of_victory"] = (df["home_score"] - df["away_score"]).abs()

#MOV multiplier
# mov_mult = log(MOV + 1) * (2.2 / ((|R_home - R_away | *0.001) + 2.2))
pre_gap = (df["pre_elo_home"] - df["pre_elo_away"]).abs()
df["mov_multiplier"] = np.log(df["margin_of_victory"] + 1.0) * (2.2 / ((pre_gap * 0.001) + 2.2))

#outcome label
df["home_win"]  = np.where(df["home_score"] > df["away_score"], 1.0,
                           np.where(df["home_score"] < df["away_score"], 0.0, 0.5))

df.to_csv(OUT_PATH, index=False)
print(f"Saved MOV-enriched dataset -> {OUT_PATH}")

# Quick spot-check
df[[
    "season","week_num","home_team","away_team",
    "home_field_advantage_pts", intl_flag_col if intl_flag_col else df.columns[0],
    "expected_home_win_prob","margin_of_victory","mov_multiplier","home_win"
]].head(12)


Saved MOV-enriched dataset -> /content/drive/MyDrive/NFL_Project/reports/games_clean_2019_2024_international_HFA_MOV.csv


,season,week_num,home_team,away_team,home_field_advantage_pts,is_international,expected_home_win_prob,margin_of_victory,mov_multiplier,home_win
0,2019,1.0,Chicago Bears,Green Bay Packers,-10.000000,0,0.485613,7.0,2.079442,0.0
1,2019,1.0,Minnesota Vikings,Atlanta Falcons,90.000000,0,0.626699,16.0,2.833213,1.0
2,2019,1.0,New York Jets,Buffalo Bills,-10.000000,0,0.485613,1.0,0.693147,0.0
3,2019,1.0,Carolina Panthers,Los Angeles Rams,-10.000000,0,0.485613,3.0,1.386294,0.0
4,2019,1.0,Cleveland Browns,Tennessee Titans,67.700510,0,0.596214,30.0,3.433987,0.0
5,2019,1.0,Jacksonville Jaguars,Kansas City Chiefs,-10.000000,0,0.485613,14.0,2.708050,0.0
6,2019,1.0,Miami Dolphins,Baltimore Ravens,90.000000,0,0.626699,49.0,3.912023,0.0
7,2019,1.0,Philadelphia Eagles,Washington Commanders,90.000000,0,0.626699,5.0,1.791759,1.0
8,2019,1.0,Seattle Seahawks,Cincinnati Bengals,25.118378,0,0.536085,1.0,0.693147,1.0
9,2019,1.0,Los Angeles Chargers,Indianapolis Colts,-10.000000,0,0.485613,6.0,1.945910,1.0


In [ ]:
df = pd.read_csv(OUT_PATH)
print("Mean of expected prob:", df["expected_home_win_prob"].mean())
print("Actual home win rate:", df["home_win"].mean())


Mean of expected prob: 0.5454371544163397
Actual home win rate: 0.5382089552238806


Goal: Establish a baseline expected-win model and MOV adjustment using the empirically derived Home Field Advantage (HFA) before running dynamic Elo updates.

Dataset

Input: games_clean_2019_2024_international_HFA.csv
Output: games_clean_2019_2024_international_HFA_MOV.csv

Key input columns:
season, week_num, home_team, away_team, home_score, away_score, home_field_advantage_pts, is_international

Method:

1.   Expected Win Probability
2.   Margin of Victory (MOV)
3.   MOV Multiplier
4.   Outcome Flag

added colums:
1. expected_home_win_prob: Baseline win probability from HFA
2. margin_of_victory: Score differential
3. mov_multiplier : Log-scaled update factor
4. home_win: Actual game outcome

Data:
- Mean expected home win prob	0.5454
- Actual home win rate	0.5382
- Difference (expected − actual)	≈ 0.7 %

result:
These results show that the empirical HFA (about +32 Elo points) reproduces the real NFL home-win advantage of roughly 54–55 %.
MOV multipliers range from 0 to 4 (log scaled), which is expected for typical score margins.


